In [2]:
# !pip install webdriver-manager # selenium

In [18]:
import re
from datetime import datetime
from math import ceil

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from tqdm.autonotebook import tqdm
from webdriver_manager.firefox import GeckoDriverManager

# CBR

In [2]:
response = requests.get("https://www.cbr.ru/banking_sector/credit/FullCoList/")
page = BeautifulSoup(response.text, "html.parser")

In [3]:
cbr_bank = []
for row in page.find_all("tr")[1:]:
    items = row.find_all("td")
    license = items[2].text
    name = items[4].text.strip()
    license_status = items[7].text.strip()
    cbr_bank.append({"license_id": license, "name": name, "status": license_status})

In [24]:
cbr_bank[:5]

[{'license_id': '2896', 'name': 'ПАО\xa0АКБ\xa0«1Банк»', 'status': 'ОТЗ'},
 {'license_id': '2306', 'name': 'АКБ\xa0«Абсолют Банк» (ПАО)', 'status': ''},
 {'license_id': '2879', 'name': 'ПАО\xa0АКБ\xa0«АВАНГАРД»', 'status': ''},
 {'license_id': '415', 'name': 'ООО\xa0Банк «Аверс»', 'status': ''},
 {'license_id': '1455', 'name': 'АО\xa0«Автоградбанк»', 'status': ''}]

In [25]:
re.sub("[\xa0\n\t]", "", cbr_bank[5]["name"])

'ООО«АвтоКредитБанк»'

## Banki.ru

In [5]:
gecko = Service(GeckoDriverManager().install())
browser = webdriver.Firefox(service=gecko)

[WDM] - ====== WebDriver manager ======
[WDM] - Current firefox version is 102.0
[WDM] - Get LATEST geckodriver version for 102.0 firefox
[WDM] - Driver [/home/samoed/.wdm/drivers/geckodriver/linux64/v0.31.0/geckodriver] found in cache


In [6]:
browser.get("https://www.banki.ru/banks/")
page = BeautifulSoup(browser.page_source, "html.parser")
page_num_text = page.find("div", class_="ui-pagination__description")
page_num = ceil(int(page_num_text.text.strip().split()[-1]) / 50)
banks = []

for i in tqdm(range(1, page_num + 1)):
    browser.get(f"https://www.banki.ru/banks/?PAGEN_1={i}")
    page = BeautifulSoup(browser.page_source, "html.parser")
    for bank in page.find_all("tr", {"data-test": "banks-list-item"}):
        bank_link = bank.find("a", class_="widget__link")
        bank_adress = bank.find("div", {"data-test": "banks-item-address"})
        license_text = bank_adress.find_all("span")[-1].text
        license = None
        if license_text.find("№") != -1:
            license = license_text.split("№")[-1].split()[0]

        banks.append(
            {
                "url": bank_link["href"].replace("/banks/", "https://www.banki.ru/services/responses/"),
                "name": bank_link.text,
                "license": license,
            }
        )

  0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
banks[:5]

[{'url': 'https://www.banki.ru/services/responses/bank/onlineipoteka/',
  'name': 'Online-Ipoteka',
  'license': '2210'},
 {'url': 'https://www.banki.ru/services/responses/bank/talkbank/',
  'name': 'TalkBank',
  'license': None},
 {'url': 'https://www.banki.ru/services/responses/bank/vkpay/',
  'name': 'VK Pay',
  'license': '3511-К'},
 {'url': 'https://www.banki.ru/services/responses/bank/absolutbank/',
  'name': 'Абсолют Банк',
  'license': '2306'},
 {'url': 'https://www.banki.ru/services/responses/bank/avangard/',
  'name': 'Авангард',
  'license': '2879'}]

In [8]:
parsed_time = datetime(2022, 7, 1)
start_time = datetime.now()

In [9]:
reviews = []
for bank in tqdm(banks):
    browser.get(bank["url"])
    page = BeautifulSoup(browser.page_source, "html.parser")

    reviews_num_elem_main = page.find("div", class_="ui-pagination__description")
    if reviews_num_elem_main is None:
        continue
    reviews_num_elem = reviews_num_elem_main.find_all("span")[1]
    reviews_num = reviews_num_elem.text.strip().split()[-1]
    page_num = ceil(int(reviews_num) / 25)

    for i in tqdm(range(1, page_num + 1), desc=bank["name"]):
        if i != 1:
            browser.get(f"{bank['url']}?page={i}")
        page = BeautifulSoup(browser.page_source, "html.parser")
        if page.find("div", class_="responses-list") is None:
            continue

        times = []
        responses = page.find("div", class_="responses-list").find_all("article")
        for response in responses:
            link = "https://www.banki.ru" + response.find("a", {"class": "header-h3"})["href"]
            text = response.find("div", class_="markup-inside-small").text
            time = datetime.strptime(response.find("time", class_="display-inline-block").text, "%d.%m.%Y %H:%M")
            comments_num = response.find("span", class_="responses__item__comment-count")
            rating_text = response.find("span", class_="rating-grade")
            rating = int(rating_text.text) if rating_text is not None else None

            if time < parsed_time:
                continue

            reviews.append(
                {
                    "bank": bank["name"],
                    "link": link,
                    "text": re.sub("\\xa0|\\n|\\t", "", text),
                    "time": time,
                    "comments_num": comments_num,
                    "rating": rating,
                }
            )
            times.append(time)

        if len(times) == 0:
            break

        if max(times) < parsed_time:
            break

  0%|          | 0/348 [00:00<?, ?it/s]

Online-Ipoteka:   0%|          | 0/1 [00:00<?, ?it/s]

TalkBank:   0%|          | 0/21 [00:00<?, ?it/s]

VK Pay:   0%|          | 0/1 [00:00<?, ?it/s]

Абсолют Банк:   0%|          | 0/133 [00:00<?, ?it/s]

...

Ю Би Эс Банк:   0%|          | 0/1 [00:00<?, ?it/s]

Юг-Инвестбанк:   0%|          | 0/2 [00:00<?, ?it/s]

ЮниКредит Банк:   0%|          | 0/369 [00:00<?, ?it/s]

Юнистрим:   0%|          | 0/10 [00:00<?, ?it/s]

Яринтербанк:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
cbr_license = [cbr["license_id"] for cbr in cbr_bank]
for cbr, bank in zip(cbr_bank, banks):
    if bank["license"] not in cbr_license:
        print(bank["license"], bank["name"])

None TalkBank
None ВЭБ
3539 Клиринговый центр МФБ
None МодульКасса
3540 РДК
